# Assignment-08 Implement of SVM, naive bayes and random forest

In [75]:
import os
import re
import scipy
import numpy as np
import pandas as pd
from pprint import pprint
from pyltp import Segmentor
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from scipy.spatial.distance import cosine
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier as KNC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV

%matplotlib inline

In [2]:
# index_col 参数不可省略，否则会出现未命名列
df = pd.read_csv('cleaned_data.csv', index_col=0)

In [3]:
# 删除 nan 无数据行
df.dropna(inplace=True)
# 重置索引为连续
df.reset_index(drop=True, inplace=True)

In [4]:
len(df)

86863

In [5]:
df.tail()

,content,label,content_cut
86858,新华社照片，多伦多，2017年6月7日\n（体育）（2）冰球——国家女子冰球队海外选秀在多伦...,1,新华社 照片 多伦多 2017年 6月 7日 体育 2 冰球 国家 女子 冰球队 海外 选秀...
86859,新华社兰州6月3日电（王衡、徐丹）记者从甘肃省交通运输厅获悉，甘肃近日集中开建高速公路、普通...,1,新华社 兰州 6月 3日 电 王衡 徐丹 记者 从 甘肃省 交通 运输厅 获悉 甘肃 近日 ...
86860,\n\n2017年5月29日，在法国巴黎郊外的凡尔赛宫，法国总统马克龙出席新闻发布会。（新华...,1,n2017 年 5月 29日 在 法国 巴黎 郊外 的 凡尔赛宫 法国 总统 马克龙 出席 ...
86861,\n\n2017年5月25日，在美国马萨诸塞州剑桥市，哈佛大学毕业生在毕业典礼上欢呼。（新华...,1,n2017 年 5月 25日 在 美国 马萨诸塞州 剑桥市 哈佛 大学 毕业生 在 毕业 典...
86862,新华社德国杜塞尔多夫６月６日电题：乒乓女球迷 \n 新华社记者王子江、张寒\n 熊老...,1,新华社 德国 杜塞尔多夫 ６月 ６日 电 题 乒乓 女 球迷 新华社 记者 王子江 张寒 熊...


### 文本向量化

In [19]:
# 指定一定数量的feature，tfidf值小于前该数量的词汇，其tfidf值设置为 0
vectorized = TfidfVectorizer(max_features=200)

In [20]:
X = vectorized.fit_transform(df.content_cut)

In [21]:
X.shape

(86863, 200)

In [66]:
df.label.value_counts()

1    78472
0     8391
Name: label, dtype: int64

* scipy库中consine的实现为：$$cosine(u, v) = 1 - \frac{u \cdot v}{||u||_2 ||v||_2}$$  取值范围[0, 2];因此这个函数的结果越小说明越相似。

In [ ]:
svc = SVC(gamma='auto')
svc.fit(Xtrain, ytrain)
svc.score(Xcv, ycv)

In [ ]:
ypred = svc.predict(Xtest)
print(classification_report(ytest, ypred))

## K-means to cluster news

## Advantages and disadvantages of classic machine learning methods
- Linear Regression

- Logistic Regression

- KNN

- Naive Bayes

- SVM

- Decision Tree

- Random Forest

- K-Means

